# Multimodal RAG

System level dependencies for `unstructured` library for parsing documents:
- `poppler`
- `tesseract`
- `libmagic`

In [1]:
# !brew install poppler tesseract libmagic

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

In [3]:
import json
from typing import List

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage

from src.app.utils import timer
from dotenv import load_dotenv
load_dotenv()

/Users/neehanth/Documents/Data Scientist/cdf_agentic_rag_chatbot_hackathon/agentic-rag-chatbot-neehanthreddym/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Partition the PDFs

In [4]:
@timer
def partition_document(file_path: str):
    """Extract elements from PDF using unstructured"""
    print(f"📃 Partitioning document: {file_path}")

    elements = partition_pdf(
        filename=file_path,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=["Image"],
        extract_image_block_to_payload=True # Convert images to base64 string if found
    )

    print(f"✅ Extracted {len(elements)} elements.")
    return elements

# Test
test_file_path = "../sample_docs/TinyLoRA_2602.04118v1.pdf"
elements = partition_document(file_path=test_file_path)

📃 Partitioning document: ../sample_docs/TinyLoRA_2602.04118v1.pdf


Loading weights: 100%|██████████| 367/367 [00:00<00:00, 1463.91it/s, Materializing param=model.query_position_embeddings.weight]                            


✅ Extracted 263 elements.
Task time: 51.50480604171753 seconds


In [5]:
# Unique elements
set([str(type(element)) for element in elements])

{"<class 'unstructured.documents.elements.FigureCaption'>",
 "<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Formula'>",
 "<class 'unstructured.documents.elements.Header'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [6]:
elements[20].to_dict()

{'type': 'Title',
 'element_id': 'd9f984638a07b1cf0c464429b1fb8a12',
 'text': '1 Introduction',
 'metadata': {'detection_class_prob': 0.8476958870887756,
  'is_extracted': 'true',
  'coordinates': {'points': ((np.float64(344.48749999999995),
     np.float64(1604.342041015625)),
    (np.float64(344.48749999999995), np.float64(1671.9508273611111)),
    (np.float64(809.7022705078125), np.float64(1671.9508273611111)),
    (np.float64(809.7022705078125), np.float64(1604.342041015625))),
   'system': 'PixelSpace',
   'layout_width': 2975,
   'layout_height': 3850},
  'last_modified': '2026-02-10T11:27:32',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '../sample_docs',
  'filename': 'TinyLoRA_2602.04118v1.pdf',
  'parent_id': 'd4ef7c2669c401106a68dfc4c46e6906'}}

In [7]:
elements[21].to_dict()

{'type': 'NarrativeText',
 'element_id': '02071c9d3a942a2b366027bf8c210501',
 'text': 'Modern language models are post-trained to "reason" in additional thinking tokens through reinforcement learning. Training for reasoning typically takes place through reinforcement learning with verifiable rewards (RLVR) and evaluated in reasoning-intensive domains such as math and coding (OpenAI et al., 2024; Shao et al., 2024; Luo et al., 2025).',
 'metadata': {'detection_class_prob': 0.9394884705543518,
  'is_extracted': 'true',
  'coordinates': {'points': ((np.float64(338.8243055555555),
     np.float64(1728.5870361328125)),
    (np.float64(338.8243055555555), np.float64(1951.6765352777777)),
    (np.float64(2633.18310546875), np.float64(1951.6765352777777)),
    (np.float64(2633.18310546875), np.float64(1728.5870361328125))),
   'system': 'PixelSpace',
   'layout_width': 2975,
   'layout_height': 3850},
  'links': [{'text': 'OpenAI et al .,',
    'url': 'cite.openai2024openaio1card',
    'start_

In [8]:
images = [element for element in elements if element.category == "Image"]
print(f"Found {len(images)} images in {test_file_path}")

Found 15 images in ../sample_docs/TinyLoRA_2602.04118v1.pdf


In [9]:
images[0].to_dict()

{'type': 'Image',
 'element_id': '347398df7639a2c5673e5b931a40ef22',
 'text': '@ aan een ee an aS Oe @@°o o Gee e @& @ ° @ ad e e,',
 'metadata': {'coordinates': {'points': ((np.float64(525.4200466666666),
     np.float64(374.8153771493052)),
    (np.float64(525.4200466666666), np.float64(905.7297354826386)),
    (np.float64(1403.6328949999997), np.float64(905.7297354826386)),
    (np.float64(1403.6328949999997), np.float64(374.8153771493052))),
   'system': 'PixelSpace',
   'layout_width': 2975,
   'layout_height': 3850},
  'last_modified': '2026-02-10T11:27:32',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 2,
  'image_base64': '...',
  'image_mime_type': 'image/jpeg',
  'file_directory': '../sample_docs',
  'filename': 'TinyLoRA_2602.04118v1.pdf'}}

In [10]:
for i, image in enumerate(images):
    img = images[i].to_dict()
    print(f"Image {i+1} url: {img["metadata"]["image_base64"]}")

Image 1 url: ...
Image 2 url: ...
Image 3 url: ...
Image 4 url: ...
Image 5 url: ...
Image 6 url: /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCADsAZ4DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3SiiiszQKKKKACiiigAooooAKKKKACsTUP+Rn0z/db+RrbrE1D/kZ9M/3W/kaBG3RRRQMKi

In [11]:
tables = [element for element in elements if element.category == "Table"]
print(f"Found {len(tables)} tables in {test_file_path}")

Found 2 tables in ../sample_docs/TinyLoRA_2602.04118v1.pdf


In [12]:
tables[0].to_dict()

{'type': 'Table',
 'element_id': '64d6671e1e78614c82b11d2c471f7d45',
 'text': 'Method Trainable Parameters Minimum FT O(nmd2) nmd2 LoRA O(nmdr) 2nmd LoRA-XS O(nmr2) nm VeRA O(nm(d + r)) 2nmd TinyLoRA O(nmu) 1',
 'metadata': {'detection_class_prob': 0.8899104595184326,
  'is_extracted': 'true',
  'coordinates': {'points': ((np.float64(983.2169189453125),
     np.float64(315.0252685546875)),
    (np.float64(983.2169189453125), np.float64(707.034423828125)),
    (np.float64(1976.4423828125), np.float64(707.034423828125)),
    (np.float64(1976.4423828125), np.float64(315.0252685546875))),
   'system': 'PixelSpace',
   'layout_width': 2975,
   'layout_height': 3850},
  'last_modified': '2026-02-10T11:27:32',
  'text_as_html': '<table><thead><tr><th>Method</th><th>Trainable Parameters</th><th>Minimum</th></tr></thead><tbody><tr><td>FT</td><td>O(nmd?)</td><td>nmd?</td></tr><tr><td>LoRA</td><td>O(nmdr)</td><td>2nmd</td></tr><tr><td>LoRA-XS</td><td>O(nmr?)</td><td>nm</td></tr><tr><td>VeRA</td><

## Chunk by title

In [13]:
@timer
def create_chunks_by_title(elements):
    """Create intelligent chunks using title-based strategy"""
    print("Creating chunks...")
    
    chunks = chunk_by_title(
        elements,
        max_characters=3000,
        new_after_n_chars=2400,
        combine_text_under_n_chars=500
    )
    
    print(f"✅ Created {len(chunks)} chunks")
    return chunks

chunks = create_chunks_by_title(elements)

Creating chunks...
✅ Created 27 chunks
Task time: 0.00374603271484375 seconds


In [14]:
chunks

In [15]:
chunks[20].to_dict()

{'type': 'CompositeElement',
 'element_id': 'da6afd6f-bd92-4464-b2cf-d5dbf5f6516f',
 'text': '10 Acknowledgements\n\nThanks to Jessy Lin, Dan Biderman, Sasha Rush, John Schulman, and Kamalika Chaudhuri for helpful feedback on this work.\n\n10',
 'metadata': {'file_directory': '../sample_docs',
  'filename': 'TinyLoRA_2602.04118v1.pdf',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2026-02-10T11:27:32',
  'page_number': 10,
  'orig_elements': 'eJzdlE9vnDAQxb+KxXlD/AcM5Ja2hyqNqirZ2ypCAx4Wa8FG2CRZRfnuNewmStu9NFIuOfKb92zP+OHNU4Qd9mh8qVV0QSIEqijSKhMqF8AboTLOk0pmCEKiaqIViXr0oMBD0D9FtbWj0gY8uuW7g72dfNmi3rY+EJGnNHiO+EEr3wbKiywNdLDa+Nm32YgkiQMSLMtiercir4AnPM5nwKhksThJDqaAIrd3Hvu5k1/6EbvbAWqMnkNBocfaa2vKugPnymG0VZDROM9SGtaIGt1hqfQYVHbczyvE8bmDfpixrV101Bjoca6utdlf25vLkkvKY5owlt+zeDiMaBb6/bAIYRg6XcO89/mxrF2Jj36E2uMydT9OGC1TMtsJtssoNxGabXS3UOfL3ird6IOcUy7PKD9jdM3YBc8uBJ/dQ3CWZuorHIOK0bltH/aZLYySy3pn7EOHartc+NLQyxnX2nfLnP7OA2ACtUgaKisKdcobQMCKFlJgVeSJ+sA88DibL7soDpf9Ah

## Process the chunks with AI summaries

In [17]:
def separate_content_types(chunk):
    """Analyze what types of content are in a chunk"""
    content_data = {
        'text': chunk.text,
        'tables': [],
        'images': [],
        'types': ['text']
    }
    
    # Check for tables and images in original elements
    if hasattr(chunk, 'metadata') and hasattr(chunk.metadata, 'orig_elements'):
        for element in chunk.metadata.orig_elements:
            element_type = type(element).__name__
            
            # Handle tables
            if element_type == 'Table':
                content_data['types'].append('table')
                table_html = getattr(element.metadata, 'text_as_html', element.text)
                content_data['tables'].append(table_html)
            
            # Handle images
            elif element_type == 'Image':
                if hasattr(element, 'metadata') and hasattr(element.metadata, 'image_base64'):
                    content_data['types'].append('image')
                    content_data['images'].append(element.metadata.image_base64)
    
    content_data['types'] = list(set(content_data['types']))
    return content_data

In [21]:
from langchain_groq import ChatGroq
from src.app.config import GROQ_MODEL, LLM_TEMPERATURE

def create_ai_summary(text: str, tables: List[str], images: List[str]) -> str:
    """Create AI-enhanced summary for mixed content"""
    try:
        llm = ChatGroq(model=GROQ_MODEL, temperature=LLM_TEMPERATURE)
        prompt = f"""You are a technical document analyst creating concise, searchable summaries for a retrieval-augmented generation (RAG) system over documents.

        Your goal is to produce a dense, keyword-rich description that maximizes retrieval relevance.

        INSTRUCTIONS:
        1. Identify the core topic, methodology, key findings, and named entities (models, datasets, metrics).
        2. Prioritize technical specificity over generic language.
        
        CONTENT TO ANALYZE:
        TEXT CONTENT:
        {text}
        """

        if tables:
            prompt += "TABLES\n"
            for i, table in enumerate(tables):
                prompt += f"Table {i+1}:\n{table}\n\n"
                prompt += """
                YOUR TASK:
                Generate a concise, searchable description covering:
                - Key facts, data points, and comparisons from text and tables
                - Main topics, methodologies, and findings
                - Visual content purpose (charts, diagrams, architecture figures)
                - Questions this content could answer
                - Alternative search terms users might use
                
                Write a single dense paragraph (150-250 words). Return ONLY the description.
                
                SEARCHABLE DESCRIPTION:
                """
        message_content = [{"type": "text", "text": prompt}]
            
        # Add images to the message
        for image_base64 in images:
            if image_base64:
                message_content.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
                })
        
        # Generate summary
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])

        return response.content or ""
    except Exception as e:
        print(f"\t❌ AI summary failed: {e}")
        # Fallback to simple summary
        summary = f"{text[:300]}..."
        if tables:
            summary += f" [Contains {len(tables)} table(s)]"
        if images:
            summary += f" [Contains {len(images)} image(s)]"
        return summary

In [19]:
import time

In [ ]:
# for element in chunks[2].metadata.orig_elements:
#     element_type = type(element).__name__
#     if element_type == 'Image':
#         print(f"Element type: {element_type}")
#         print(f"image_base64: {element.metadata.image_base64}")

Element type: Image
image_base64: ...
Element type: Image
image_base64: ...


In [22]:
@timer
def summarise_chunks(chunks):
    """Process all chunks with AI Summaries"""
    print(" Processing chunks with AI Summaries...")
    
    langchain_documents = []
    total_chunks = len(chunks)

    for i, chunk in enumerate(chunks):
        current_chunk = i + 1
        print(f"\tProcessing chunk {current_chunk}/{total_chunks}")
        
        # Analyze chunk content
        content_data = separate_content_types(chunk)
        print(f"\t\tContent types found: {content_data['types']}")
        print(f"\t\tTables: {len(content_data['tables'])} | Images: {len(content_data['images'])}")

        # Summarize the chunks (only if the chunk has tables and/or images)
        if content_data['tables'] or content_data['images']:
            print('\tSummarizing the mixed content...')
            try:
                content_summary = create_ai_summary(
                    content_data['text'],
                    content_data['tables'],
                    content_data['images']
                )
                print(f"\t\t✅ Summarizing the content successfully")
                print(f"\t\tSummarize content: {(content_summary or '')[:100]}...")
            except Exception as e:
                print(f"\t\t❌ AI summary failed: {e}")
                content_summary = content_data['text']
            time.sleep(2) # rate-limit handler
        else:
            print(f"\t\tUsing raw text (No Tables or Images found)")
            content_summary = content_data['text']
        
        content_summary = content_summary or content_data['text'] or ""
        
        langchain_doc = Document(
            page_content=content_summary,
            metadata={
                "original_content": json.dumps({
                    "raw_text": content_data['text'],
                    "tables_html": content_data['tables'],
                    "images_base64": content_data['images']
                })
            }
        )
        langchain_documents.append(langchain_doc)

    print(f"✅ Processed {len(langchain_documents)} chunks")
    return langchain_documents

# Process chunks with AI Summaries
processed_chunks = summarise_chunks(chunks)

 Processing chunks with AI Summaries...
	Processing chunk 1/27
		Content types found: ['text']
		Tables: 0 | Images: 0
		Using raw text (No Tables or Images found)
	Processing chunk 2/27
		Content types found: ['text']
		Tables: 0 | Images: 0
		Using raw text (No Tables or Images found)
	Processing chunk 3/27
		Content types found: ['image', 'text']
		Tables: 0 | Images: 2
	Summarizing the mixed content...
		✅ Summarizing the content successfully
		Summarize content: Here is a dense, keyword-rich description that maximizes retrieval relevance:

**Core Topic:** Effic...
	Processing chunk 4/27
		Content types found: ['image', 'text']
		Tables: 0 | Images: 2
	Summarizing the mixed content...
		✅ Summarizing the content successfully
		Summarize content: Here is a dense, keyword-rich description that maximizes retrieval relevance:

**Core Topic:** Param...
	Processing chunk 5/27
		Content types found: ['text']
		Tables: 0 | Images: 0
		Using raw text (No Tables or Images found)
	Processing 

In [24]:
# Verify image summarization coverage
total_image_chunks = 0
summarized_image_chunks = 0

for i, (chunk, doc) in enumerate(zip(chunks, processed_chunks)):
    content_data = separate_content_types(chunk)
    if content_data['images']:
        total_image_chunks += 1
        # If AI summary worked, page_content will differ from raw text
        if doc.page_content != content_data['text']:
            summarized_image_chunks += 1
            print(f"  ✅ Chunk {i+1}: {len(content_data['images'])} image(s) — AI summarized")
        else:
            print(f"  ❌ Chunk {i+1}: {len(content_data['images'])} image(s) — fell back to raw text")

print(f"\n📊 Image Summary Coverage: {summarized_image_chunks}/{total_image_chunks} chunks with images were AI-summarized")

  ✅ Chunk 3: 2 image(s) — AI summarized
  ✅ Chunk 4: 2 image(s) — AI summarized
  ✅ Chunk 15: 5 image(s) — AI summarized
  ✅ Chunk 18: 2 image(s) — AI summarized
  ✅ Chunk 19: 2 image(s) — AI summarized
  ✅ Chunk 27: 2 image(s) — AI summarized

📊 Image Summary Coverage: 6/6 chunks with images were AI-summarized


In [25]:
# Verify table summarization coverage
total_table_chunks = 0
summarized_table_chunks = 0

for i, (chunk, doc) in enumerate(zip(chunks, processed_chunks)):
    content_data = separate_content_types(chunk)
    if content_data['tables']:
        total_table_chunks += 1
        # If AI summary worked, page_content will differ from raw text
        if doc.page_content != content_data['text']:
            summarized_table_chunks += 1
            print(f"  ✅ Chunk {i+1}: {len(content_data['tables'])} table(s) — AI summarized")
        else:
            print(f"  ❌ Chunk {i+1}: {len(content_data['tables'])} table(s) — fell back to raw text")

print(f"\n📊 Table Summary Coverage: {summarized_table_chunks}/{total_table_chunks} chunks with tables were AI-summarized")

  ✅ Chunk 9: 1 table(s) — AI summarized
  ✅ Chunk 14: 1 table(s) — AI summarized

📊 Table Summary Coverage: 2/2 chunks with tables were AI-summarized


In [27]:
def export_chunks_to_json(chunks, filename="chunks_export.json"):
    """Export processed chunks to clean JSON format"""
    export_data = []
    
    for i, doc in enumerate(chunks):
        chunk_data = {
            "chunk_id": i + 1,
            "enhanced_content": doc.page_content,
            "metadata": {
                "original_content": json.loads(doc.metadata.get("original_content", "{}"))
            }
        }
        export_data.append(chunk_data)
    
    # Save to file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Exported {len(export_data)} chunks to {filename}")
    return export_data

# Export your chunks
json_data = export_chunks_to_json(processed_chunks)

✅ Exported 27 chunks to chunks_export.json


## Create and persist ChromaDB

In [29]:
from src.app.config import EMBEDDING_MODEL

@timer
def create_vector_store(documents, persist_directory="dbv1/chroma_db"):
    """Create and persist ChromaDB vector store"""
    print("🔮 Creating embeddings and storing in ChromaDB...")
        
    embedding_model = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
    
    # Create ChromaDB vector store
    print("--- Creating vector store ---")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_directory, 
        collection_metadata={"hnsw:space": "cosine"}
    )
    print("--- Finished creating vector store ---")
    
    print(f"✅ Vector store created and saved to {persist_directory}")
    return vectorstore

# Create the vector store
db = create_vector_store(processed_chunks)

🔮 Creating embeddings and storing in ChromaDB...
--- Creating vector store ---
--- Finished creating vector store ---
✅ Vector store created and saved to dbv1/chroma_db
Task time: 2.560701847076416 seconds


In [ ]:
# Retrieval
query = "According to the paper, why is Reinforcement Learning (RL) more effective than Supervised Finetuning (SFT) in the low-parameter regime?"
retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

# Export to JSON
export_chunks_to_json(chunks, "retriever_results.json")

✅ Exported 3 chunks to retriever_results.json


[{'chunk_id': 1,
  'enhanced_content': 'TinyLoRA is a technique for reducing the number of trainable parameters in large language models, achieving as few as 1 changed parameter. Compared to existing methods, including LoRA, LoRA-XS, and VeRA, TinyLoRA significantly decreases parameter usage: O(nmu) vs O(nmdr), O(nmr), and O(nm(d+r)), respectively. By replacing the r × r matrix R with a low-dimensional trainable vector v ∈ Ru projected through a fixed random tensor P ∈ Ru×r×r, TinyLoRA enables training with a single parameter when all modules share weights. The method allows for weight tying across modules, reducing total trainable parameters to O(nmu/ntie). In a transformer architecture like LLaMA-3 70B with 80 layers, TinyLoRA reduces the parameter count from 560 to as few as 1. Key comparisons are summarized in Table 1, highlighting TinyLoRA\'s advantages over FT, LoRA, LoRA-XS, and VeRA in terms of trainable parameters and minimum requirements. This technique is relevant for users 

## Complete Ingestion Pipeline

In [31]:
@timer
def run_complete_ingestion_pipeline(pdf_path: str):
    """Run the complete RAG ingestion pipeline"""
    print("🚀 Starting RAG Ingestion Pipeline")
    print("=" * 50)
    
    # 1: Partition
    elements = partition_document(pdf_path)
    
    # 2: Chunk
    chunks = create_chunks_by_title(elements)
    
    # 3: AI Summarisation
    summarised_chunks = summarise_chunks(chunks)
    
    # 4: Vector Store
    db = create_vector_store(summarised_chunks, persist_directory="dbv2/chroma_db")
    
    print("🎉 Pipeline completed successfully!")
    return db

In [33]:
db = run_complete_ingestion_pipeline("../sample_docs/TinyLoRA_2602.04118v1.pdf")

🚀 Starting RAG Ingestion Pipeline
📃 Partitioning document: ../sample_docs/TinyLoRA_2602.04118v1.pdf
✅ Extracted 263 elements.
Task time: 47.04861640930176 seconds
Creating chunks...
✅ Created 27 chunks
Task time: 0.00177001953125 seconds
 Processing chunks with AI Summaries...
	Processing chunk 1/27
		Content types found: ['text']
		Tables: 0 | Images: 0
		Using raw text (No Tables or Images found)
	Processing chunk 2/27
		Content types found: ['text']
		Tables: 0 | Images: 0
		Using raw text (No Tables or Images found)
	Processing chunk 3/27
		Content types found: ['image', 'text']
		Tables: 0 | Images: 2
	Summarizing the mixed content...
		✅ Summarizing the content successfully
		Summarize content: Here is a dense, keyword-rich description that maximizes retrieval relevance:

**Core Topic:** Effic...
	Processing chunk 4/27
		Content types found: ['image', 'text']
		Tables: 0 | Images: 2
	Summarizing the mixed content...
		✅ Summarizing the content successfully
		Summarize content: He

In [49]:
# Query the vector store
query = "In the precision ablation study for updates under 1KB, which data format was found to be the most performant bit-for-bit? A.bf16 B.int8 C.fp16 D.fp32"

retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

@timer
def generate_final_answer(chunks, query):
    """Generate final answer using multimodal content"""
    
    try:
        # Initialize LLM (needs vision model for images)
        llm = ChatGroq(model=GROQ_MODEL, temperature=LLM_TEMPERATURE)
        
        # Build the text prompt
        prompt_text = f"""Based on the following documents, please answer this question: {query}

        CONTENT TO ANALYZE:
        """
        
        for i, chunk in enumerate(chunks):
            prompt_text += f"--- Document {i+1} ---\n"
            
            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])
                
                # Add raw text
                raw_text = original_data.get("raw_text", "")
                if raw_text:
                    prompt_text += f"TEXT:\n{raw_text}\n\n"
                
                # Add tables as HTML
                tables_html = original_data.get("tables_html", [])
                if tables_html:
                    prompt_text += "TABLES:\n"
                    for j, table in enumerate(tables_html):
                        prompt_text += f"Table {j+1}:\n{table}\n\n"
            
            prompt_text += "\n"
        
        prompt_text += """
        Please provide a clear, comprehensive answer using the text, tables, and images above. If the documents don't contain sufficient information to answer the question, say "I don't have enough information to answer that question based on the provided documents and do not answer with your own knowledge."

        ANSWER:
        """

        message_content = [{"type": "text", "text": prompt_text}]
        
        # Add all images from all chunks
        for chunk in chunks:
            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])
                images_base64 = original_data.get("images_base64", [])
                
                for image_base64 in images_base64:
                    message_content.append({
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
                    })
        
        # Send to AI and get response
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])
        
        return response.content
        
    except Exception as e:
        print(f"❌ Answer generation failed: {e}")
        return "Sorry, I encountered an error while generating the answer."

# Usage
final_answer = generate_final_answer(chunks, query)
print(final_answer)

Task time: 1.6518399715423584 seconds
## Step 1: Analyze the provided documents to find relevant information about the precision ablation study for updates under 1KB.
The documents provided contain information about a study on scaling TinyLoRA, a method for efficient fine-tuning of large language models. The focus is on the performance of different data formats under strict update size constraints, specifically under 1KB.

## 2: Identify the specific data formats mentioned for the precision ablation study.
The data formats mentioned are bf16, int8, fp16, and fp32.

## 3: Locate the specific results for the precision ablation study under 1KB update size.
According to Document 1 and as visually represented in the figures, particularly in Figure 4 and related texts, it is mentioned that "fp32 outperforms bf16 and float16 even when accounting for its twice-as-large update size in bytes." This implies a comparison of performance across different precisions.

## 4: Determine which data forma